In [ ]:
!pip install nb-black > /dev/null
!pip install mutagen > /dev/null

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mutagen
from mutagen.ogg import OggFileType
from tqdm.notebook import tqdm

plt.style.use("ggplot")

%load_ext lab_black

In [ ]:
train = pd.read_csv("../input/birdclef-2022/train_metadata.csv")
scored = pd.read_json("../input/birdclef-2022/scored_birds.json")

# is_scored
scored["is_scored"] = True
scored.rename({0: "label"}, axis=1, inplace=True)
train = (
    pd.merge(train, scored, left_on="primary_label", right_on="label", how="left")
    .fillna(False)
    .drop("label", axis=1)
)

# num_secondary_labels
train["num_secondary_labels"] = train["secondary_labels"].apply(lambda x: len(eval(x)))

In [ ]:
data = {
    "length": [],
    "channels": [],
    "bitrate": [],
    "sample_rate": [],
}

for item in tqdm(train.itertuples(), total=len(train)):
    file_path = item.filename
    f = mutagen.File(f"../input/birdclef-2022/train_audio/{file_path}")
    for key in data.keys():
        data[key].append(getattr(f.info, key))
audio_meta = pd.DataFrame(data)
audio_meta.to_csv("audio_metadata.csv", index=False)

In [ ]:
audio_meta.describe()

In [ ]:
sns.histplot(audio_meta[["length"]], log_scale=True)
plt.title("Histogram of Audio Length")

In [ ]:
audio_meta["channels"].value_counts().plot(kind="bar")
plt.xlabel("Channels")
plt.ylabel("Count")
plt.title("Histogram of Audio Channels")

In [ ]:
audio_meta["bitrate"].value_counts().plot(kind="bar")
plt.xlabel("BitRate")
plt.ylabel("Count")
plt.title("Histogram of Audio Bitrates")

# Output Extended Train Metadata

In [ ]:
train_ext = pd.concat([train, audio_meta], axis=1)
train_ext.to_csv("train_ext.csv", index=False)